## SelectFeatures
This notebook shows the development of how we select important features from `ChemFeatures` to study with. We used the following method to analyze the weight of each feature:

* LASSO (Least Absolute Shrinkage and Selection Operator)
* RFE (Recursive Feature Elimination)
* RIDGE (Ridge Regression)
* SS (Stability Selection)

We use explained variance to compare and evaluate these methods for final decision making.<br>

#### Use ``ChemFeatures`` to generate a dataframe of features.

In [1]:
import pandas as pd
# Module for extracting ChemInfo
from SeekFeatures import ChemFeatures
import mordred
import scikit
# plotting package
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Use 100K data for demonstration
data = pd.read_csv('..\Database\HCEPD_100K.csv') 
# Use first 50 data to illustrate the analysis
data50 = data.head(50)
data50.head()

,id,SMILES_str,stoich_str,mass,pce,voc,jsc,e_homo_alpha,e_gap_alpha,e_lumo_alpha,tmp_smiles_str
0,655365,C1C=CC=C1c1cc2[se]c3c4occc4c4nsnc4c3c2cn1,C18H9N3OSSe,394.3151,5.161953,0.867601,91.567575,-5.467601,2.022944,-3.444656,C1=CC=C(C1)c1cc2[se]c3c4occc4c4nsnc4c3c2cn1
1,1245190,C1C=CC=C1c1cc2[se]c3c(ncc4ccccc34)c2c2=C[SiH2]...,C22H15NSeSi,400.4135,5.261398,0.504824,160.401549,-5.104824,1.630750,-3.474074,C1=CC=C(C1)c1cc2[se]c3c(ncc4ccccc34)c2c2=C[SiH...
2,21847,C1C=c2ccc3c4c[nH]cc4c4c5[SiH2]C(=Cc5oc4c3c2=C1...,C24H17NOSi,363.4903,0.000000,0.000000,197.474780,-4.539526,1.462158,-3.077368,C1=CC=C(C1)C1=Cc2oc3c(c2[SiH2]1)c1c[nH]cc1c1cc...
3,65553,[SiH2]1C=CC2=C1C=C([SiH2]2)C1=Cc2[se]ccc2[SiH2]1,C12H12SeSi3,319.4448,6.138294,0.630274,149.887545,-5.230274,1.682250,-3.548025,C1=CC2=C([SiH2]1)C=C([SiH2]2)C1=Cc2[se]ccc2[Si...
4,720918,C1C=c2c3ccsc3c3[se]c4cc(oc4c3c2=C1)C1=CC=CC1,C20H12OSSe,379.3398,1.991366,0.242119,126.581347,-4.842119,1.809439,-3.032680,C1=CC=C(C1)c1cc2[se]c3c4sccc4c4=CCC=c4c3c2o1


Generate a dataframe of chemical features.

In [3]:
features_df = ChemFeatures(data50['SMILES_str'])

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  7.06it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  7.40it/s]


In [4]:
features_df.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,20.142136,16.169815,0,0,33.204238,2.616114,5.057172,33.204238,1.383510,4.188377,...,10.460213,77.559765,394.963154,11.968580,1216,42,146.0,185.0,4.611111,5.000000
1,20.849242,16.133746,0,0,34.755190,2.600656,5.047921,34.755190,1.390208,4.221130,...,10.493799,77.890806,401.013897,10.025347,1337,45,150.0,189.0,4.861111,5.250000
2,22.889683,17.893320,0,0,37.437727,2.645420,5.160050,37.437727,1.386582,4.315006,...,10.639862,82.032579,363.107941,8.252453,1635,49,168.0,215.0,5.083333,5.555556
3,13.313708,11.688393,0,0,21.283525,2.481194,4.637583,21.283525,1.330220,3.777767,...,9.782393,66.966647,319.941201,11.426471,430,20,94.0,115.0,3.166667,3.333333
4,19.435029,15.989365,0,0,31.670490,2.627835,5.062549,31.670490,1.376978,4.154569,...,10.429310,77.119188,379.977407,10.856497,1074,39,142.0,181.0,4.361111,4.750000


> Not all molecules have the same features as others, so there will be some non-value entries hidden in the dataframe.

In [8]:
# find the non-value entries.

missing =[]
for i in range(features_df.shape[1]):
    if type(features_df.loc[1][i]) == mordred.error.Missing:
        missing.append(features_df.loc[1][i])

# show examples of the non-value entries.        
missing[0:5]

In [60]:
import numpy as np
assd = features_df.dtypes
type(assd[1])

numpy.dtype

> **for these non-value entries, we assign them to 0, simply indicating that the molecule does not possess that specific feature.**

In [91]:
type(0.5)

float

In [102]:
# replace missing value(wrong type) with 0
type_series = features_df.dtypes
wrong_column = []
for col in range(len(type_series)):
    if type_series[col] != np.dtype('int64') and type_series[col] != np.dtype('float64'):
        wrong_column.append(col)
for column in wrong_column:
    for item in features_df.iloc[:,column]:
        if type(item) != float and type(item) != int:
            features_df.iloc[:,column] = features_df.iloc[:,column].replace(item, 0)
np.unique(type_series)

array([dtype('bool'), dtype('int64'), dtype('float64'), dtype('O')],
      dtype=object)

In [ ]:
from sklearn.preprocessing import StandardScaler
features =[feature for feature in features_df.columns]

# Separating out the features
x = features_df.loc[:, features].values

# Our target is pce
y = features_df.loc[:, 'pce'].values
sc = StandardScaler()
X = sc.fit_transform(x)

In [ ]:
features = [ftr for ftr in features_df.head()]
ranks = {}
def rank_to_dict(ranks, names, order=1):
    sc = StandardScaler()
    ranks = sc.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
estimatorREFCV = SVR('linear')
selectorREFCV = RFECV(estimatorREFCV, step=5, scoring='explained_variance')
selectorREFCV.fit(X, y)

In [ ]:
ridge = Ridge(alpha=7)
ridge.fit(X, Y)
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)